In [1]:
import os

In [2]:
%pwd

'c:\\Users\\deena\\Sales Project\\Revenue-Prediction-MLOps\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\deena\\Sales Project\\Revenue-Prediction-MLOps'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:

import os
from src.mlproject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def preprocess_data(self,data):
        
        data['Date'] = pd.to_datetime(data['Date'])
        data['Month'] = data['Date'].dt.month
        data.drop('Date', axis=1, inplace=True)
        categorical_features = ['Product Category', 'Region', 'Payment Method', 'Product Name']
        for feature in categorical_features:
            le = preprocessing.LabelEncoder()
            data[feature] = le.fit_transform(data[feature])
        return data
    
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        # Preprocess data df_preprocessed
        data_preprocessed = self.preprocess_data(data.drop('Total Revenue', axis=1))
        data_preprocessed['Total Revenue'] = data['Total Revenue']

        # Split the data into training and test sets (0.75, 0.25) split
        train, test = train_test_split(data_preprocessed, test_size=0.2, random_state=42)

        # Save the split data to CSV files
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        # Log the shapes of the split data
        logger.info("Split data into training and test sets")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print(train.shape)
        print(test.shape)


In [10]:

if __name__ == "__main__":
    try:
        config = ConfigurationManager()
        data_transformation_config = config.get_data_transformation_config()
        data_transformation = DataTransformation(config=data_transformation_config)
        data_transformation.train_test_spliting()
        
    except Exception as e:
        logger.error("An error occurred during data transformation: %s", e)
        raise e

[2024-06-06 14:54:42,465: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-06 14:54:42,466: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-06 14:54:42,467: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-06 14:54:42,470: INFO: common: created directory at: artifacts]
[2024-06-06 14:54:42,471: INFO: common: created directory at: artifacts/data_transformation]
[2024-06-06 14:54:42,486: INFO: 3643743089: Split data into training and test sets]
[2024-06-06 14:54:42,487: INFO: 3643743089: Train shape: (192, 9)]
[2024-06-06 14:54:42,487: INFO: 3643743089: Test shape: (48, 9)]
(192, 9)
(48, 9)
